In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'current_date={current_date}')

current_date=2023-05-24 08:00:00


In [6]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/47255
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2023-04-26 08:00:00 to 2023-05-24 07:00:00


2023-05-24 05:58:03,945 INFO: USE `taxi_demand_ytjqdg_featurestore`
2023-05-24 05:58:04,401 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_demand_ytjqdg_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2023-04-25 08:00:00.000' AND `fg0`.`pickup_hour` <= TIMESTAMP '2023-05-25 07:00:00.000'


In [7]:
features.head()

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id
166,0.0,1.0,2.0,1.0,0.0,0.0,2.0,8.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2023-05-24 08:00:00,1
250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-24 08:00:00,2
54,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-24 08:00:00,3
71,6.0,10.0,4.0,2.0,1.0,3.0,8.0,2.0,3.0,5.0,...,3.0,0.0,1.0,1.0,1.0,0.0,1.0,9.0,2023-05-24 08:00:00,4
69,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-05-24 08:00:00,5


In [8]:
from src.inference import load_model_from_registry, get_model_predictions

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/47255
Connected. Call `.close()` to terminate connection gracefully.

In [10]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,1.0,2023-05-24 08:00:00
1,2,0.0,2023-05-24 08:00:00
2,3,0.0,2023-05-24 08:00:00
3,4,7.0,2023-05-24 08:00:00
4,5,0.0,2023-05-24 08:00:00
...,...,...,...
260,261,18.0,2023-05-24 08:00:00
261,262,104.0,2023-05-24 08:00:00
262,263,112.0,2023-05-24 08:00:00
263,264,52.0,2023-05-24 08:00:00


### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [11]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/47255
Connected. Call `.close()` to terminate connection gracefully.


In [12]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/47255/fs/46149/fg/51640


Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

%3|1684919201.698|FAIL|Joaos-MacBook-Pro-2.local#producer-1| [thrd:ssl://c.app.hopsworks.ai:9092/bootstrap]: ssl://c.app.hopsworks.ai:9092/bootstrap: SSL handshake failed: error:1416F086:SSL routines:tls_process_server_certificate:certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (brew install openssl) (after 109ms in state SSL_HANDSHAKE)
%3|1684919202.677|FAIL|Joaos-MacBook-Pro-2.local#producer-1| [thrd:ssl://3.145.101.110:9092/bootstrap]: ssl://3.145.101.110:9092/bootstrap: SSL handshake failed: error:1416F086:SSL routines:tls_process_server_certificate:certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (brew install openssl) (after 97ms in state SSL_HANDSHAKE)
%3|1684919203.697|FAIL|Joaos-MacBook-Pro-2.local#producer-1| [thrd:ssl://3.142.251.253:9092/bootstrap]: ssl://3

KeyboardInterrupt: 

%3|1684919510.602|FAIL|Joaos-MacBook-Pro-2.local#producer-1| [thrd:ssl://c.app.hopsworks.ai:9092/bootstrap]: ssl://c.app.hopsworks.ai:9092/bootstrap: SSL handshake failed: error:1416F086:SSL routines:tls_process_server_certificate:certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (brew install openssl) (after 111ms in state SSL_HANDSHAKE, 10 identical error(s) suppressed)
%3|1684919524.618|FAIL|Joaos-MacBook-Pro-2.local#producer-1| [thrd:ssl://3.142.251.253:9092/bootstrap]: ssl://3.142.251.253:9092/bootstrap: SSL handshake failed: error:1416F086:SSL routines:tls_process_server_certificate:certificate verify failed: broker certificate could not be verified, verify that ssl.ca.location is correctly configured or root CA certificates are installed (brew install openssl) (after 98ms in state SSL_HANDSHAKE, 8 identical error(s) suppressed)
%3|1684919531.649|FAIL|Joaos-MacBook-Pro-2.l